In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [2]:
iOption0 = 0 # piminus, electron

In [3]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-100M_piminus_single_hagedorn_04-5_v0.root","my-10M_electron_embed_helios_v3.root"]
file_name=file_names[iOption0]
outfilenames= ['pionminus','electron']#output
filename=outfilenames[iOption0]
part_ids=[9,3]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]

In [4]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [6]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [7]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.8 and myevent.GetEntry(ipart).GetN0()>0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
              round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
              myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
              round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
              round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

99748438
Nevent: 1377, centrality: 18.0, x: 0.3367440104484558, Ntracks: 1
pt:  0.453 , e/p:  0.831 , n0:  1 , disp:  9.11 , chi2/npe0:  0.41 , prob:  0.031 , id:  -pion
0.45760464668273926
Nevent: 1949, centrality: 10.0, x: 0.344310998916626, Ntracks: 1
pt:  0.453 , e/p:  0.881 , n0:  1 , disp:  6.25 , chi2/npe0:  75.0 , prob:  0.556 , id:  -pion
0.4554281532764435
Nevent: 3154, centrality: 56.0, x: 0.3541480004787445, Ntracks: 1
pt:  0.539 , e/p:  0.908 , n0:  2 , disp:  7.05 , chi2/npe0:  23.77 , prob:  0.908 , id:  -pion
1.123041033744812
Nevent: 3568, centrality: 29.0, x: 0.3390769958496094, Ntracks: 1
pt:  0.474 , e/p:  0.952 , n0:  2 , disp:  6.61 , chi2/npe0:  6.96 , prob:  0.006 , id:  -pion
0.5532205700874329
Nevent: 3695, centrality: 6.0, x: 0.33977600932121277, Ntracks: 1
pt:  0.256 , e/p:  1.053 , n0:  3 , disp:  2.05 , chi2/npe0:  6.33 , prob:  0.822 , id:  -pion
1.8461459875106812
Nevent: 4129, centrality: 53.0, x: 0.3340129852294922, Ntracks: 1
pt:  0.739 , e/p:  0.837 

In [8]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60000000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 and myevent.GetEntry(ipart).GetN0()>=0):
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 and myevent.GetEntry(ipart).GetN0()>=0):
            data_array.append([int(myevent.GetCentrality()),myevent.GetPreciseZ(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),myevent.GetEntry(ipart).GetChi2(),myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetPhi0Prime()   ])


data = pd.DataFrame(data_array, columns=('centrality','zvertex','pt', 'e/p', 'n0', 'disp', 'chi2', 'npe0', 'prob', 'phi0'))

99748438


In [9]:
data.head()

,centrality,zvertex,pt,e/p,n0,disp,chi2,npe0,prob,phi0
0,54,6.14125,0.505566,0.688748,1,4.444584,80.354012,2,0.660573,3.289515
1,64,-1.98948,0.414231,0.634064,2,9.015498,13.140485,2,0.002139,3.085341
2,16,3.49265,1.491351,0.613041,0,340.027008,23.499977,0,0.000005,3.333536
3,18,-1.10746,0.452858,0.831287,1,9.109297,0.811236,2,0.030966,3.392909
4,19,5.60962,0.742303,1.083055,0,313.606201,43.685745,0,0.000007,2.862729


In [10]:
print(data.size, data.shape, good_event_counter)

3480120 (348012, 10) 346554


In [11]:
filepath = '/home/yoren/bnl/PHENIX/ee/ML/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [12]:
data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)